#+NAME: AlexNet
#+YEAR: 2012
#+AUTHORS: Alex Krishevsky, Ilya Sutskever, Geoffrey Hinton
#+CATEGORIES: Convolution
#+DESCRIPTION: The model that kickstarted the deep learning revolution. Consisting of eight layers, including five convolutional layers and three fully connected layers. AlexNet was one of the first deep convolutional neural networks to achieve state-of-the-art results on ImageNet. The original implementation split the network over two independent gpus to alleviate challenges with memory at the time.
#+PAPER: /static/alexnet.pdf
#+IMAGE: /static/alexnet.png

# Overview

The model that kickstarted the deep learning revolution. Consisting of eight layers, including five convolutional layers and three fully connected layers. AlexNet was one of the first deep convolutional neural networks to achieve state-of-the-art results on ImageNet. The original implementation split the network over two independent gpus to alleviate challenges with memory at the time.

In [ ]:
use burn::{
  config::Config,
  module::Module,
  nn::{
    conv::{Conv2d, Conv2dConfig},
    PaddingConfig2d,
    pool::{
      MaxPool2d, MaxPool2dConfig, 
      AdaptiveAvgPool2d, AdaptiveAvgPool2dConfig
    },
    Linear, LinearConfig, 
    Dropout, DropoutConfig,
    ReLU,
  },
  tensor::{backend::Backend, Tensor},
};

#[derive(Module, Debug)]
pub struct Model<B: Backend> {
  conv1: Conv2d<B>,
  conv2: Conv2d<B>,
  conv3: Conv2d<B>,
  conv4: Conv2d<B>,
  conv5: Conv2d<B>,
  maxpool: MaxPool2d,
  avgpool: AdaptiveAvgPool2d,
  linear1: Linear<B>,
  linear2: Linear<B>,
  linear3: Linear<B>,
  dropout: Dropout,
  activation: ReLU
}

#[derive(Config, Debug)]
pub struct ModelConfig {
  num_classes: usize,
  #[config(default = "0.5")]
  dropout:f64
}

impl ModelConfig {
  pub fn init<B: Backend>(&self, device: &B::Device) -> Model<B> {
    Model {
      conv1: Conv2dConfig::new([3, 64], [11, 11])
        .with_stride([4, 4])
        .with_padding(PaddingConfig2d::Same)
        .init(device),
      conv2: Conv2dConfig::new([64, 192], [5, 5])
        .with_padding(PaddingConfig2d::Same)
        .init(device),
      conv3: Conv2dConfig::new([192, 384], [3, 3])
        .with_padding(PaddingConfig2d::Same)
        .init(device),
      conv4: Conv2dConfig::new([384, 256], [3, 3])
        .with_padding(PaddingConfig2d::Same)
        .init(device),
      conv5: Conv2dConfig::new([256, 256], [3, 3])
        .with_padding(PaddingConfig2d::Same)
        .init(device),
      maxpool: MaxPool2dConfig::new([3, 3])
        .with_strides([2, 2])
        .init(),
      avgpool: AdaptiveAvgPool2dConfig::new([6, 6]).init(),
      linear1: LinearConfig::new(256*6*6, 4096).init(device),
      linear2: LinearConfig::new(4096, 4096).init(device),
      linear3: LinearConfig::new(4096, self.num_classes).init(device),
      dropout: DropoutConfig::new(self.dropout).init(),
      activation: ReLU::new(),
    }
  }

  pub fn init_with<B: Backend>(&self, record: ModelRecord<B>) -> Model<B> {
    Model {
      conv1: Conv2dConfig::new([3, 64], [11, 11]).init_with(record.conv1),
      conv2: Conv2dConfig::new([64, 192], [5, 5]).init_with(record.conv2),
      conv3: Conv2dConfig::new([192, 384], [3, 3]).init_with(record.conv3),
      conv4: Conv2dConfig::new([384, 256], [3, 3]).init_with(record.conv4),
      conv5: Conv2dConfig::new([256, 256], [3, 3]).init_with(record.conv5),
      maxpool: MaxPool2dConfig::new([3, 3]).init(),
      avgpool: AdaptiveAvgPool2dConfig::new([6, 6]).init(),
      linear1: LinearConfig::new(256*6*6, 4096).init_with(record.linear1),
      linear2: LinearConfig::new(4096, 4096).init_with(record.linear2),
      linear3: LinearConfig::new(4096, self.num_classes).init_with(record.linear3),
      dropout: DropoutConfig::new(self.dropout).init(),
      activation: ReLU::new(),
    }
  }
}

impl<B: Backend> Model<B> {
  pub fn forward(&self, x: Tensor<B, 4>) -> Tensor<B, 2> {
    let [batch_size, _channels, _height, _width] = x.dims();

    let x = self.conv1.forward(x);
    let x = self.activation.forward(x);
    let x = self.maxpool.forward(x);

    let x = self.conv2.forward(x);
    let x = self.activation.forward(x);
    let x = self.maxpool.forward(x);

    let x = self.conv3.forward(x);
    let x = self.activation.forward(x);

    let x = self.conv4.forward(x);
    let x = self.activation.forward(x);

    let x = self.conv5.forward(x);
    let x = self.activation.forward(x);
    let x = self.maxpool.forward(x);

    let x = self.avgpool.forward(x);
    let x = x.reshape([batch_size, 256*6*6]);
    
    let x = self.dropout.forward(x);
    let x = self.linear1.forward(x);
    let x = self.activation.forward(x);

    let x = self.dropout.forward(x);
    let x = self.linear2.forward(x);
    let x = self.activation.forward(x);

    self.linear3.forward(x)
  }
}